In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [23]:
df_enc = pd.read_csv("data/enc/data-enc-2024-04-22.csv")

In [24]:
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105783 entries, 0 to 105782
Data columns (total 86 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   avg_price_5                       105783 non-null  float64
 1   bathrooms_total                   105783 non-null  float64
 2   bedrooms_extra                    105783 non-null  float64
 3   bedrooms                          105783 non-null  float64
 4   stories_total                     105783 non-null  float64
 5   size_interior                     105783 non-null  float64
 6   lng                               105783 non-null  float64
 7   lat                               105783 non-null  float64
 8   parkings                          105783 non-null  float64
 9   price                             105783 non-null  float64
 10  household_income                  105783 non-null  float64
 11  individual_income                 105783 non-null  f

In [25]:
df_ON = df_enc[df_enc["province_Ontario"] == True]
df_QC = df_enc[df_enc["province_Quebec"] == True]
df_BC = df_enc[df_enc["province_British Columbia"] == True]
df_AB = df_enc[df_enc["province_Alberta"] == True]
df_SK = df_enc[df_enc["province_Saskatchewan"] == True]
df_MB = df_enc[df_enc["province_Manitoba"] == True]
# East
df_ES = df_enc[(df_enc["province_Nova Scotia"] == True) |
               (df_enc["province_New Brunswick"] == True) |
               (df_enc["province_Newfoundland & Labrador"] == True) |
               (df_enc["province_Prince Edward Island"] == True)]
# North
df_NO = df_enc[(df_enc["province_Yukon"] == True) |
               (df_enc["province_Northwest Territories"] == True)]

df_ON = df_ON.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_QC = df_QC.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_BC = df_BC.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_AB = df_AB.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_SK = df_SK.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_MB = df_MB.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta", "province_Saskatchewan",
                    "province_Manitoba", "province_Nova Scotia", "province_New Brunswick", "province_Newfoundland & Labrador",
                    "province_Prince Edward Island", "province_Yukon", "province_Northwest Territories"], axis=1)
df_ES = df_ES.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta",
                    "province_Saskatchewan","province_Manitoba", "province_Yukon", "province_Northwest Territories"], axis=1)
df_NO = df_NO.drop(["province_Ontario", "province_Quebec", "province_British Columbia", "province_Alberta",
                    "province_Saskatchewan", "province_Manitoba", "province_Nova Scotia", "province_New Brunswick",
                    "province_Newfoundland & Labrador", "province_Prince Edward Island"], axis=1)

In [26]:
df_ON.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48140 entries, 0 to 102605
Data columns (total 74 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   avg_price_5                    48140 non-null  float64
 1   bathrooms_total                48140 non-null  float64
 2   bedrooms_extra                 48140 non-null  float64
 3   bedrooms                       48140 non-null  float64
 4   stories_total                  48140 non-null  float64
 5   size_interior                  48140 non-null  float64
 6   lng                            48140 non-null  float64
 7   lat                            48140 non-null  float64
 8   parkings                       48140 non-null  float64
 9   price                          48140 non-null  float64
 10  household_income               48140 non-null  float64
 11  individual_income              48140 non-null  float64
 12  commute_transit                48140 non-null  flo

In [27]:
df_ON.corr(numeric_only=True)["price"].sort_values(ascending=False).head(20)

price                           1.000000
avg_price_5                     0.679840
size_interior                   0.527034
bathrooms_total                 0.525510
household_income                0.467328
individual_income               0.467328
bedrooms                        0.384252
parkings                        0.352119
building_type_House             0.276204
single_family                   0.264292
edu_bachelor_degree             0.258293
ownership_type_group_ids_1.0    0.249451
edu_university_certificate      0.238346
stories_total                   0.234883
edu_post_graduate_degree        0.217938
owners                          0.208117
bedrooms_extra                  0.206492
age_15_to_19                    0.176104
multi_family                    0.163873
area_single_detached            0.153695
Name: price, dtype: float64

## Raw Input

In [28]:
kf = KFold(n_splits=10)

In [29]:
cb1 = CatBoostRegressor(silent=True)

X = df_ON.drop("price", axis=1)
y = df_ON["price"]

scores1 = cross_val_score(cb1, X, y, cv=kf, scoring="r2")

print(f"\n{scores1}\n")
pd.Series(scores1).describe()


[0.61563188 0.72722323 0.72811576 0.47921945 0.69762061 0.72032555
 0.7217129  0.70549091 0.73077913 0.70829408]



count    10.000000
mean      0.683441
std       0.079294
min       0.479219
25%       0.699588
50%       0.714310
75%       0.725846
max       0.730779
dtype: float64

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cb1.fit(X_train, y_train)
y_pred = cb1.predict(X_test)

In [31]:
r2_score(y_test, y_pred)

0.7386521520233383

In [32]:
root_mean_squared_error(y_test, y_pred)

283732.8165818224

In [33]:
y_df1 = pd.DataFrame({"y_test":y_test.values, "y_pred":y_pred})

In [34]:
np.round(y_df1.head(20))

,y_test,y_pred
0,788000.0,866394.0
1,1500000.0,1563935.0
2,485000.0,529651.0
3,959900.0,972363.0
4,1099000.0,1048357.0
5,2498800.0,2334312.0
6,1249900.0,1046893.0
7,2199900.0,1868270.0
8,575000.0,593760.0
9,799000.0,681779.0


## Scaled

In [35]:
cb2 = CatBoostRegressor(silent=True)

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1,1))

scores2 = cross_val_score(cb2, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores2}\n")
pd.Series(scores2).describe()


[0.6166587  0.72838753 0.72669084 0.48095162 0.708044   0.71257141
 0.72031917 0.69834996 0.73146368 0.70275954]



count    10.000000
mean      0.682620
std       0.078217
min       0.480952
25%       0.699452
50%       0.710308
75%       0.725098
max       0.731464
dtype: float64

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb2.fit(X_train, y_train)
y_pred = cb2.predict(X_test)

In [37]:
r2_score(y_test, y_pred)

0.7379608479689655

In [38]:
root_mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_pred.reshape(-1,1)))

284107.8265744813

In [39]:
y_df2 = pd.DataFrame({"y_test":scaler_y.inverse_transform(y_test[:,0].reshape(-1,1))[:,0].tolist(),
                      "y_pred":scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist()})

In [40]:
np.round(y_df2.head(20))

,y_test,y_pred
0,788000.0,864578.0
1,1500000.0,1488860.0
2,485000.0,532721.0
3,959900.0,957020.0
4,1099000.0,1049699.0
5,2498800.0,2272979.0
6,1249900.0,1009594.0
7,2199900.0,1848521.0
8,575000.0,605869.0
9,799000.0,669910.0


## log1p: X Only

In [41]:
from scipy import stats

sub_df = df_AB.copy(deep=True)

skew_df = pd.DataFrame(sub_df.columns, columns=["Feature"])
skew_df["Corr_Price_Before"] = sub_df.corr()["price"].values
skew_df["Skew_Before"] = skew_df["Feature"].apply(lambda feature: stats.skew(sub_df[feature]))
skew_df["AbsSkew_Before"] = skew_df["Skew_Before"].apply(abs)
skew_df["Skewed_Before"] = skew_df["AbsSkew_Before"].apply(lambda x: True if x>= 0.5 else False)

# before_log = set(skew_df.query("Skewed_Before == True")["Feature"].values)

for column in skew_df.query("Skewed_Before == True")["Feature"].values:
    if column != "price":
        sub_df[column] = np.log1p(sub_df[column])


# skew_df = pd.DataFrame(sub_df.columns, columns=["Feature"])
skew_df["Corr_Price_After"] = sub_df.corr()["price"].values
skew_df["Skew_After"] = skew_df["Feature"].apply(lambda feature: stats.skew(sub_df[feature]))
skew_df["AbsSkew_After"] = skew_df["Skew_After"].apply(abs)
skew_df["Skewed_After"] = skew_df["AbsSkew_After"].apply(lambda x: True if x>= 0.5 else False)

# after_log = set(skew_df.query("Skewed_Before == True")["Feature"].values)

# log_columns = list(before_log - after_log - {"price", "lng", "lat"})
log_columns = skew_df[(skew_df["Skewed_Before"]==True) & (skew_df["Corr_Price_After"] > skew_df["Corr_Price_Before"])]["Feature"].values

log_columns

/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


array(['bedrooms_extra', 'commute_transit', 'commute_drive',
       'age_65_to_79', 'lang_en_only', 'lang_en_and_fr',
       'area_apt_5_plus_floors', 'loc_transit_friendly', 'loc_wellness',
       'ownership_type_group_ids_0.0', 'ownership_type_group_ids_2.0'],
      dtype=object)

In [42]:
cb3 = CatBoostRegressor(silent=True)

X_log = X.copy(deep=True)
X_log[log_columns] = np.log1p(X_log[log_columns])

X_scaled = scaler_X.fit_transform(X_log)
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1,1))

scores3 = cross_val_score(cb3, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores3}\n")
pd.Series(scores3).describe()


[0.6166587  0.72838753 0.72669084 0.48095162 0.708044   0.71257141
 0.72031917 0.69834999 0.73146368 0.70275958]



count    10.000000
mean      0.682620
std       0.078217
min       0.480952
25%       0.699452
50%       0.710308
75%       0.725098
max       0.731464
dtype: float64

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb3.fit(X_train, y_train)
y_pred = cb3.predict(X_test)

In [44]:
r2_score(y_test, y_pred)

0.7379608479689655

In [45]:
root_mean_squared_error(y_test, y_pred)

0.5289526995450891

In [46]:
y_df3 = pd.DataFrame({"y_test":scaler_y.inverse_transform(y_test[:,0].reshape(-1,1))[:,0].tolist(),
                      "y_pred":scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist()})

In [47]:
np.round(y_df3.head(20))

,y_test,y_pred
0,788000.0,864578.0
1,1500000.0,1488860.0
2,485000.0,532721.0
3,959900.0,957020.0
4,1099000.0,1049699.0
5,2498800.0,2272979.0
6,1249900.0,1009594.0
7,2199900.0,1848521.0
8,575000.0,605869.0
9,799000.0,669910.0


# log1p: X and y

In [48]:
cb4 = CatBoostRegressor(silent=True)

X_log = X.copy(deep=True)
X_log[log_columns] = np.log1p(X_log[log_columns])

y_log = np.log1p(y)

X_scaled = scaler_X.fit_transform(X_log)
y_scaled = scaler_y.fit_transform(y_log.to_numpy().reshape(-1,1))

scores4 = cross_val_score(cb4, X_scaled, y_scaled, cv=kf, scoring="r2")

print(f"\n{scores4}\n")
pd.Series(scores4).describe()


[0.74603385 0.85021156 0.80407366 0.68100484 0.77341089 0.78751598
 0.84338915 0.79549002 0.81137359 0.80508041]



count    10.000000
mean      0.789758
std       0.048856
min       0.681005
25%       0.776937
50%       0.799782
75%       0.809800
max       0.850212
dtype: float64

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

cb4.fit(X_train, y_train)
y_pred = cb4.predict(X_test)

In [50]:
r2_score(y_test, y_pred)

0.8184391828296314

### Reverse Log

In [51]:
r2_score(np.expm1(y_test), np.expm1(y_pred))

0.4067042111270702

In [52]:
root_mean_squared_error(y_test, y_pred)

0.4322421069109251

In [53]:
y_df4 = pd.DataFrame({"y_test":np.expm1(scaler_y.inverse_transform(y_test))[:,0].tolist(),
                      "y_pred":np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))[:,0].tolist())})

In [54]:
np.round(y_df4.head(20))

,y_test,y_pred
0,788000.0,821116.0
1,1500000.0,1522831.0
2,485000.0,505593.0
3,959900.0,919521.0
4,1099000.0,952402.0
5,2498800.0,2190748.0
6,1249900.0,1003634.0
7,2199900.0,1743642.0
8,575000.0,578337.0
9,799000.0,673319.0


In [55]:
np.round(y_df3.head(20))

,y_test,y_pred
0,788000.0,864578.0
1,1500000.0,1488860.0
2,485000.0,532721.0
3,959900.0,957020.0
4,1099000.0,1049699.0
5,2498800.0,2272979.0
6,1249900.0,1009594.0
7,2199900.0,1848521.0
8,575000.0,605869.0
9,799000.0,669910.0


In [56]:
plt.figure(figsize=(12,8))
plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_test)))
plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))

/tmp/ipykernel_2748/305300287.py:2: RuntimeWarning: overflow encountered in expm1
  plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_test)))
/tmp/ipykernel_2748/305300287.py:3: RuntimeWarning: overflow encountered in expm1
  plt.scatter(np.expm1(scaler_X.inverse_transform(X_test))[:,5], np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))


/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/matplotlib/ticker.py:2096: RuntimeWarning: overflow encountered in multiply
  steps = self._extended_steps * scale
/home/soheil/codes/House-Pred-CA/House-Pred-CA/.venv/lib64/python3.12/site-packages/matplotlib/ticker.py:2131: RuntimeWarning: overflow encountered in multiply
  ticks = np.arange(low, high + 1) * step + best_vmin


Error in callback <function _draw_all_if_interactive at 0x7f972552cd60> (for post_execute), with arguments args (),kwargs {}:


OverflowError: cannot convert float infinity to integer

OverflowError: cannot convert float infinity to integer

<Figure size 1200x800 with 1 Axes>

In [57]:
r2_score(np.expm1(scaler_y.inverse_transform(y_test)), np.expm1(scaler_y.inverse_transform(y_pred.reshape(-1,1))))

0.7331382984115143

In [58]:
X_log

,avg_price_5,bathrooms_total,bedrooms_extra,bedrooms,stories_total,size_interior,lng,lat,parkings,household_income,...,loc_cafes,loc_quiet,loc_parks,building_type_Apartment,building_type_House,building_type_Row / Townhouse,building_type_Semi-Detached,ownership_type_group_ids_0.0,ownership_type_group_ids_1.0,ownership_type_group_ids_2.0
0,495739.8,3.0,0.000000,4.0,2.5,2810.000000,-83.010728,42.321421,0.0,138252.2947,...,3.0,5.0,3.5,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
1,634900.0,2.0,0.000000,2.0,1.0,1072.000000,-83.029633,42.265783,0.0,125972.0819,...,1.5,4.5,3.5,1.0,0.0,0.0,0.0,0.0,0.0,0.693147
2,1009739.8,3.0,0.000000,4.0,2.0,1965.881818,-83.087577,42.228649,0.0,205459.7539,...,1.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
3,349180.0,1.0,0.000000,4.0,1.0,1040.426146,-83.056441,42.304613,0.0,76724.5170,...,2.5,3.0,3.5,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
4,315360.0,1.0,0.000000,3.0,1.0,3718.455676,-83.055635,42.302468,0.0,78194.1950,...,2.0,4.0,3.5,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102262,609900.0,3.0,1.098612,2.0,2.0,2597.906470,-77.506600,46.105039,8.0,157800.6433,...,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
102263,412920.0,1.0,0.000000,2.0,1.0,974.999891,-77.453295,46.012428,4.0,123004.6681,...,0.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
102603,311920.0,2.0,0.000000,2.0,1.0,1270.000000,-89.281918,48.370675,4.0,101012.3023,...,2.5,4.5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000
102604,351940.0,2.0,0.000000,2.0,1.0,1150.000000,-89.186733,48.457044,0.0,91878.4614,...,1.0,2.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.693147


In [59]:
y

0         849900.0
1         399900.0
2         959900.0
3         299900.0
4         399000.0
            ...   
102262    769900.0
102263    379900.0
102603    339000.0
102604    349900.0
102605    299900.0
Name: price, Length: 48140, dtype: float64

In [60]:
df_log = X_log.copy(deep=True)
df_log["price_log"] = np.log1p(y)

In [61]:
df_log.corr(numeric_only=True)["price_log"].sort_values(ascending=False).head(20)

price_log                       1.000000
avg_price_5                     0.698764
bathrooms_total                 0.618783
size_interior                   0.576967
household_income                0.467815
individual_income               0.467815
bedrooms                        0.460364
parkings                        0.419148
single_family                   0.359504
building_type_House             0.337451
ownership_type_group_ids_1.0    0.324011
edu_bachelor_degree             0.288137
owners                          0.285262
stories_total                   0.281920
edu_university_certificate      0.261922
bedrooms_extra                  0.260368
multi_family                    0.230089
edu_post_graduate_degree        0.224621
age_15_to_19                    0.219192
area_single_detached            0.192651
Name: price_log, dtype: float64

In [68]:
X.columns

Index(['avg_price_5', 'bathrooms_total', 'bedrooms_extra', 'bedrooms',
       'stories_total', 'size_interior', 'lng', 'lat', 'parkings',
       'household_income', 'individual_income', 'commute_transit',
       'commute_foot', 'commute_bicycle', 'commute_drive', 'single_family',
       'multi_family', 'single_person', 'multi_person', 'total_individuals',
       'age_0_to_4', 'age_5_to_9', 'age_10_to_14', 'age_15_to_19',
       'age_20_to_34', 'age_35_to_49', 'age_50_to_64', 'age_65_to_79',
       'age_80_plus', 'owners', 'renters', 'lang_en_only', 'lang_fr_only',
       'lang_en_and_fr', 'lang_other', 'edu_no_high_school', 'edu_high_school',
       'edu_trade_certificate', 'edu_college_certificate',
       'edu_university_certificate', 'edu_bachelor_degree',
       'edu_post_graduate_degree', 'household_children',
       'area_single_detached', 'area_semi_detached', 'area_duplex',
       'area_row_houses', 'area_apt_1_to_4_floors', 'area_apt_5_plus_floors',
       'loc_high_schools', 